In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#modules
#import libraries
# Familiar imports
import numpy as np
import pandas as pd

# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# For training random forest model
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import mean_squared_error

# Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno


# Configure visualisations
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set(context="notebook", palette="dark", style = 'whitegrid' , color_codes=True)

# Data preprocessing :
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, scale, LabelEncoder, OneHotEncoder

from sklearn import preprocessing

from xgboost import XGBRegressor


In [ ]:
#import the dataset
train =pd.read_csv('/content/drive/MyDrive/analytics_comp/TRAIN (1).csv')
test =pd.read_csv('/content/drive/MyDrive/analytics_comp/TEST_FINAL.csv')
sample_submission =pd.read_csv('/content/drive/MyDrive/analytics_comp/SAMPLE.csv')

#lengths 188340 22265

In [ ]:
#feature engineering

df = train.copy()
df_test = test.copy()
#extract week day out of date
df['Day']=df['Date'].apply(lambda x :pd.to_datetime(x,format = '%Y-%m-%d').dayofweek )
df_test['Day']= df_test['Date'].apply(lambda x :pd.to_datetime(x,format = '%Y-%m-%d').dayofweek )

In [ ]:
#target encoding for Store_id
col ='Store_id'
feat = train.groupby(col)["Sales"].agg("mean")
df['tar_enc_Store_id']= df[col].map(feat)
df_test['tar_enc_Store_id']=df_test[col].map(feat)

In [ ]:
#kfolds
from sklearn import model_selection
kf = model_selection.KFold(n_splits=5,shuffle=True,random_state=42)
df['kfold']=-1
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=df)):
    df.loc[valid_indicies, "kfold"] = fold


In [ ]:
useful_features=['Store_Type', 'Location_Type', 'Region_Code', 'Holiday',
       'Discount','Day']
ord_cols=[ 'Region_Code','Discount'] 
one_hot =['Store_Type', 'Location_Type']
predictions=[]
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest =  df_test.copy()

    ytrain = xtrain.Sales
    yvalid = xvalid.Sales
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    xtest  = xtest[useful_features]

    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[ord_cols] = ordinal_encoder.fit_transform(xtrain[ord_cols])
    xvalid[ord_cols] = ordinal_encoder.transform(xvalid[ord_cols])
    xtest[ord_cols] = ordinal_encoder.transform(xtest[ord_cols])

    #onehot cols
    # Apply one-hot encoder to each column with categorical data
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(xtrain[one_hot]))
    OH_cols_valid = pd.DataFrame(OH_encoder.transform(xvalid[one_hot]))
    OH_cols_test  = pd.DataFrame(OH_encoder.transform(xtest[one_hot]))

    # One-hot encoding removed index; put it back
    OH_cols_train.index = xtrain.index
    OH_cols_valid.index = xvalid.index

    # Remove categorical columns (will replace with one-hot encoding)
    num_X_train = xtrain.drop(one_hot, axis=1)
    num_X_valid = xvalid.drop(one_hot, axis=1)
    num_X_test  = xtest.drop(one_hot,axis=1)

    # Add one-hot encoded columns to numerical features
    xtrain = pd.concat([num_X_train, OH_cols_train], axis=1)
    xvalid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
    xtest  = pd.concat([num_X_test, OH_cols_test],   axis=1)




    model = RandomForestRegressor(random_state=1,n_estimators=1000)
    # Train the model (will take about 10 minutes to run)
    model.fit(xtrain,ytrain)
    preds_valid = model.predict(xvalid)
    predictions.append(model.predict(xtest)) # Splits
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)


0 11410.708892789644
1 11480.509288378862
2 11409.15982118347
3 11377.441516119608
4 11372.305882842757


In [ ]:
model = RandomForestRegressor(random_state=1,n_estimators=1000)
def complete(df,df_test,model):
  useful_features=['Store_Type', 'Location_Type', 'Region_Code', 'Holiday',
        'Discount','d']
  ord_cols=[ 'Region_Code','Discount'] 
  one_hot =['Store_Type', 'Location_Type']
  predictions=[]
  xtrain =  df.copy()
  xtest =  df_test.copy()
  xtrain['d'] = xtrain['Date'].apply(lambda x :pd.to_datetime(x,format = '%Y-%m-%d').day) 
  xtest['d']  = xtest['Date'].apply(lambda x :pd.to_datetime(x,format = '%Y-%m-%d').day)

  ytrain = xtrain.Sales

  xtrain = xtrain[useful_features]
  xtest  = xtest[useful_features]

  ordinal_encoder = preprocessing.OrdinalEncoder()
  xtrain[ord_cols] = ordinal_encoder.fit_transform(xtrain[ord_cols])
  xtest[ord_cols] = ordinal_encoder.transform(xtest[ord_cols])

  #onehot cols
  # Apply one-hot encoder to each column with categorical data
  OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
  OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(xtrain[one_hot]))
  OH_cols_test  = pd.DataFrame(OH_encoder.transform(xtest[one_hot]))

  # One-hot encoding removed index; put it back
  OH_cols_train.index = xtrain.index

  # Remove categorical columns (will replace with one-hot encoding)
  num_X_train = xtrain.drop(one_hot, axis=1)
  num_X_test  = xtest.drop(one_hot,axis=1)

  # Add one-hot encoded columns to numerical features
  xtrain = pd.concat([num_X_train, OH_cols_train], axis=1)
  xtest  = pd.concat([num_X_test, OH_cols_test],   axis=1)

  # Train the model (will take about 10 minutes to run)
  model.fit(xtrain,ytrain)
  preds_train = model.predict(xtrain)
  rmse = mean_squared_error(yvalid, preds_valid, squared=False)
  predictions.append(model.predict(xtest)) # Splits
  #rmse = mean_squared_error(yvalid, preds_valid, squared=False)
  #print(fold, rmse)
  return rmse,predictions[0]


In [ ]:
xgb_params = {
'lambda': 67.79737006663706,
'alpha': 40.12405005448161,
'colsample_bytree': 0.061613774851329205,
'subsample': 0.9556736521337416,
'learning_rate': 0.17024722721525629,
'n_estimators': 9489,
'max_depth':3,
'booster': 'gbtree',
'min_child_weight': 155,
'seed' : 38,
    #'tree_method':'gpu_hist'
}
model= XGBRegressor()
rmse,result = complete(df,df_test,model)
print(rmse)

[13:45:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
11372.305882842757


In [ ]:
len(submission)

22265

In [ ]:
result[:20]

array([62596.63 , 39418.01 , 63503.367, 38271.723, 38793.   , 63503.367,
       62596.63 , 48801.574, 53448.86 , 33482.79 , 53688.406, 38793.   ,
       38511.273, 43158.547, 44065.285, 34855.168, 49322.848, 23826.459,
       39418.01 , 54595.145], dtype=float32)

In [ ]:
sample_submission.Sales = submission
sample_submission.to_csv("submission9.csv", index=False)

In [ ]:
train.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52
